# Creating Subarray Images with SPyFFI
This notebook shows some of the basics of SPyFFI, using it to create images over four CCDs for one TESS camera.

In [1]:
# this sets where to look for inputs and store outputs 
# (it will default to `~/.tess/spyffi` if the $SPYFFIDATA environment variable isn't set)
import os
os.environ["SPYFFIDATA"] = '/Users/zkbt/Cosmos/Data/TESS/FFIs'

In [2]:
# this makes sure that updates are actually printed to the terminal (instead of just to a log)
import logging
logging.basicConfig(level="INFO")

In [3]:
# this loads a default observation definition, which is a dictionary of dictionaries
from SPyFFI.Observation import Observation, default
inputs = default

The inputs are all organized into dictionaries within the `inputs` dictionary.

In [4]:
# how do we define the catalog of stars to use (and the light curves)
print default['catalog']

{'skykw': {'faintlimit': None, 'fast': False}, 'catalogue': 'UCAC4', 'starsarevariable': True, 'name': 'testpattern', 'lckw': {'fractionwithextremelc': 0.005, 'fractionofstarswithlc': 0.5, 'fractionwithcustom': 0.1, 'fractionwithrotation': 0.2, 'seed': 0, 'fainteststarwithlc': None, 'options': ['trapezoid', 'sin'], 'fractionwithtrapezoid': 0.3}, 'testpatternkw': {'randomizenudgesby': 21.1, 'randomizepropermotionsby': 0.0, 'spacing': 500.0, 'magnitudes': [6, 16], 'randomizemagnitudes': True}}


In [5]:
# how should exposures be created (what should be written out?)
print default['expose']

{'magnitudethreshold': 999, 'jitter': True, 'correctcosmics': True, 'smear': False, 'cosmicsdiffusion': True, 'compress': {120: True, 1800: False, 2: True, 20: True}, 'cosmicsversion': 'fancy', 'writecosmics': True, 'writesimulated': True, 'skipcosmics': False, 'writenoiseless': False, 'display': False}


In [6]:
# set up the geometry of the camera, and (importantly) the PSF library
print default['camera']

{'psfkw': {'nsubpixelsperpixel': 101, 'focus_toinclude': [0, 10], 'npixels': 21, 'noffsets_toinclude': 11, 'npositions_toinclude': 11, 'version': 'RRUasbuilt', 'stellartemp_toinclude': [4350], 'debprefix': 'woods_prf_feb2016/RAYS_ptSrc_wSi_Oct27model_AP40.6_75C_F3p314adj'}, 'focuskw': {'span': [0.0, 10.0]}, 'positionangle': 0.0, 'label': 'subarray', 'jitterkw': {'amplifyinterexposurejitter': 1.0, 'rawjitterbasename': 'cartoon.jitter', 'jitterrms': None}, 'stamps': {120: None, 1800: None, 2: None, 20: None}, 'variablefocus': False, 'counterstep': 1, 'warpspaceandtime': False, 'subarray': 200, 'dec': 66.56070833333332, 'roll': 0, 'cadence': 1800, 'ra': 270.0}


In [7]:
# in what ways should we jitter the motion of the camera?
print default['jitter']

{'amplifyinterexposurejitter': 1.0, 'rawjitterbasename': 'cartoon.jitter', 'jitterrms': None}


In [8]:
# how many of each exposure time should we create (and in what order?)
print default['observation']

{'cadencestodo': {2: 100}, 'collate': True}


Let's modify some of those input options, by changing values associated with some of the directory keys inside each sub-group.

In [10]:
# give a label to this observation (that goes into the directory name)
inputs['camera']['label'] = ''
# how big of a subarray in pixels (centered on the FOV); None will give four normal CCDs
inputs['camera']['subarray'] = None
# should we change the focus throughout the orbit? 
inputs['camera']['variablefocus'] = False

# should we skip injecting cosmic rays? (False = *do* inject cosmics)
inputs['expose']['skipcosmics'] = False
# let's write the cosmic ray images out as separate files
inputs['expose']['writecosmics'] = True

# what's the catalog ("testpattern" or "UCAC4")
inputs['catalog']['name'] = 'testpattern'

# how many of each exposure duration should we make?
inputs['observation']['cadencestodo'] = {1800:3, 120:3, 2:3}
o = Observation(inputs)

INFO:SPyFFI.Observation:  creating a new observation, with the following inputs:
INFO:SPyFFI.Observation:      inputs[expose] = 
INFO:SPyFFI.Observation:                 magnitudethreshold:999
INFO:SPyFFI.Observation:                             jitter:True
INFO:SPyFFI.Observation:                     correctcosmics:True
INFO:SPyFFI.Observation:                              smear:False
INFO:SPyFFI.Observation:                   cosmicsdiffusion:True
INFO:SPyFFI.Observation:                           compress:{120: True, 1800: False, 2: True, 20: True}
INFO:SPyFFI.Observation:                     cosmicsversion:fancy
INFO:SPyFFI.Observation:                       writecosmics:True
INFO:SPyFFI.Observation:                     writesimulated:True
INFO:SPyFFI.Observation:                        skipcosmics:False
INFO:SPyFFI.Observation:                     writenoiseless:False
INFO:SPyFFI.Observation:                            display:False
INFO:SPyFFI.Observation:      inputs[camera] = 


In [ ]:
o.create()

INFO:SPyFFI.Camera:setting cadence to 1800 seconds = 900 reads.
INFO:SPyFFI.Jitter:populating the jitterball for 1800 second cadence, based on the raw jitter file cartoon.jitter.cadence1800s.unscaled.
INFO:SPyFFI.PSF:initializing PSF painter, based on woods_prf_feb2016/RAYS_ptSrc_wSi_Oct27model_AP40.6_75C_F3p314adj
INFO:SPyFFI.Intrapixel:creating an intrapixel sensitivity map called perfectpixels
INFO:SPyFFI.PSF:setting up the pixel arrays
INFO:SPyFFI.PSF:created pixel coordinate arrays
INFO:SPyFFI.CCD:created CCD #1, of size 21x21
INFO:SPyFFI.PSF:trying to load PSFs from /Users/zkbt/Cosmos/Data/TESS/FFIs/intermediates/psfs/RRUasbuilt/focus0and10_stellartemp4350/pixelizedlibrary_cartoon.jitter.cadence1800s.unscaled_perfectpixels_11positions_11offsets.npy
INFO:SPyFFI.PSF:...success!
INFO:SPyFFI.Catalogs:projecting catalog -0.799 years relative to 2018
INFO:SPyFFI.Catalogs:save projected star catalog /Users/zkbt/Cosmos/Data/TESS/FFIs/outputs/testpattern_6to16/1800s/ccd1/catalog_testpatte

This should now take a while (like a few minutes?) to create a bunch of images and drop them in `$SPYFFIDATA/testpattern_6to16/`. Go check them out and see what's what!